In [1]:
# Import libraries
import numpy as np
import pandas as pd

# LEER CHAT GPT SOBRE STEP BY STEP EN CLEANING Y VER SI LA COLUMNA AÑOS QUE CREE CAPTURA LOS AÑOS QUE SEÑALAN  YEAR_WITH NANS PARA VER SI CALZA

In [3]:
# Import hospital discharges from 2021 to 2024
df = pd.read_parquet(r"C:\Users\mirko\Desktop\Curso Data Science DL\ProyectoEDA_Egresos_Hospitalarios_2001-2020\data\processed\Discharges_2021-2024_messy.parquet")

In [4]:
# A general idea of the rows and columns
df.head()

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
0,No Pertenecientes al Sistema Nacional de Servi...,1,1 A 4 AÑOS,ISAPRE,2021,K590,1,1,2021
1,No Pertenecientes al Sistema Nacional de Servi...,1,1 A 4 AÑOS,NINGUNA,2021,S015,1,1,2021
2,No Pertenecientes al Sistema Nacional de Servi...,1,1 A 4 AÑOS,ISAPRE,2021,T181,1,1,2021
3,No Pertenecientes al Sistema Nacional de Servi...,1,1 A 4 AÑOS,FONASA,2021,Q381,1,1,2021
4,No Pertenecientes al Sistema Nacional de Servi...,1,1 A 4 AÑOS,FONASA,2021,Q531,1,1,2021


In [5]:
# shape of the dataframe
df.shape                         

(6343796, 9)

In [6]:
# Datatypes are as light as possible
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6343796 entries, 0 to 1667348
Data columns (total 9 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   Healthcare facility type  category
 1   Sex                       category
 2   Age                       category
 3   Health insurance          category
 4   Year_with_nans            category
 5   Primary diagnosis code    category
 6   Length of stay            int32   
 7   Discharge condition       category
 8   Year                      int16   
dtypes: category(7), int16(1), int32(1)
memory usage: 133.4 MB


In [7]:
# The only numeric columns is length of stay
df["Length of stay"].describe()  

count    6.343796e+06
mean     6.247676e+00
std      4.637814e+01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      6.000000e+00
max      2.223900e+04
Name: Length of stay, dtype: float64

In [8]:
# Columns in the dataframe
df.columns

Index(['Healthcare facility type', 'Sex', 'Age', 'Health insurance',
       'Year_with_nans', 'Primary diagnosis code', 'Length of stay',
       'Discharge condition', 'Year'],
      dtype='object')

In [9]:
# Apparently the missingness of the data is structured
df.isna().sum()

Healthcare facility type    1651836
Sex                          173892
Age                          167677
Health insurance             173892
Year_with_nans               167677
Primary diagnosis code            0
Length of stay                    0
Discharge condition               0
Year                              0
dtype: int64

In [10]:
# Only the "Healthcare facility type" column has many null values (around 26%), but is important to understand the dinamic of the missing values
df.isna().mean()

Healthcare facility type    0.260386
Sex                         0.027411
Age                         0.026432
Health insurance            0.027411
Year_with_nans              0.026432
Primary diagnosis code      0.000000
Length of stay              0.000000
Discharge condition         0.000000
Year                        0.000000
dtype: float64

# Handling missing data and values on each column: EXPLORAR QUE LA MISSIGNESS NO ES RANDOM, ESTA ESTRCTURADA

Healthcare facility type

In [13]:
# The facilities are public (SNSS) or private (not SNSS)
df["Healthcare facility type"].value_counts()

Healthcare facility type
Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS       2779010
No Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS    1912950
Name: count, dtype: int64

In [14]:
# Out of 1651836 rows that have missing healthcare facility type, there are 167677 patients without identified sex, insurance, age and year of discharge
df.loc[df['Healthcare facility type'].isnull()].isna().sum()

Healthcare facility type    1651836
Sex                          167677
Age                          167677
Health insurance             167677
Year_with_nans               167677
Primary diagnosis code            0
Length of stay                    0
Discharge condition               0
Year                              0
dtype: int64

In [15]:
# Always when age, sex, health insurance or year_with_nans are missing, there are not valid values in healthcare facility type and the rest of the variables
# This is proven beacuse the dataframes are empty
print(df[df['Healthcare facility type'].notna() & df['Age'].isna() & df['Sex'].notna() & df['Health insurance'].notna() & df['Year_with_nans'].notna()])
print(df[df['Healthcare facility type'].notna() & df['Age'].notna() & df['Sex'].isna() & df['Health insurance'].notna() & df['Year_with_nans'].notna()])
print(df[df['Healthcare facility type'].notna() & df['Age'].notna() & df['Sex'].notna() & df['Health insurance'].isna() & df['Year_with_nans'].notna()])
print(df[df['Healthcare facility type'].notna() & df['Age'].notna() & df['Sex'].notna() & df['Health insurance'].notna() & df['Year_with_nans'].isna()])

Empty DataFrame
Columns: [Healthcare facility type, Sex, Age, Health insurance, Year_with_nans, Primary diagnosis code, Length of stay, Discharge condition, Year]
Index: []
Empty DataFrame
Columns: [Healthcare facility type, Sex, Age, Health insurance, Year_with_nans, Primary diagnosis code, Length of stay, Discharge condition, Year]
Index: []
Empty DataFrame
Columns: [Healthcare facility type, Sex, Age, Health insurance, Year_with_nans, Primary diagnosis code, Length of stay, Discharge condition, Year]
Index: []
Empty DataFrame
Columns: [Healthcare facility type, Sex, Age, Health insurance, Year_with_nans, Primary diagnosis code, Length of stay, Discharge condition, Year]
Index: []


In [16]:
# By the other hand, there are rows that that having missing healthcare facility type are valid in the rest of the columns.
# It does not happen in the opposite way around as shown above
df[df['Healthcare facility type'].isna() & df['Age'].notna() & df['Sex'].notna() & df['Health insurance'].notna() & df['Year_with_nans'].notna()]

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
0,NaN,HOMBRE,1 a 9,FONASA,2023,D180,7,1,2023
1,NaN,HOMBRE,1 a 9,FONASA,2023,T181,2,1,2023
2,NaN,HOMBRE,1 a 9,FONASA,2023,K358,1,1,2023
3,NaN,HOMBRE,1 a 9,FONASA,2023,K353,6,1,2023
4,NaN,HOMBRE,1 a 9,FONASA,2023,K353,1,1,2023
...,...,...,...,...,...,...,...,...,...
1484154,NaN,HOMBRE,menor de un año,FONASA,2023,Q249,2,1,2023
1484155,NaN,HOMBRE,menor de un año,FONASA,2023,D807,3,1,2023
1484156,NaN,HOMBRE,menor de un año,FONASA,2023,A499,4,1,2023
1484157,NaN,HOMBRE,menor de un año,FONASA,2023,T182,1,1,2023


In [17]:
# The missing values for healthcare facility type when the rest of the columns are valid can follow a certaint pattern.
# In this case, always when the healthcare facility type is missing, the year of discharge was 2023
df[df['Healthcare facility type'].isna() & df['Age'].notna() & df['Sex'].notna() & df['Health insurance'].notna() & df['Year_with_nans'].notna()]["Year_with_nans"].value_counts()

Year_with_nans
2023    1484159
2021          0
2022          0
2024          0
Name: count, dtype: int64

In [18]:
# Healthcare facility type is missing for the year 2023 or when the year or age is also missing.
# 167677 (year or age is missing) + 1484159 (year is 2023) = 1651836 (total amount of NaNs for healthvare facility type column)
df[df['Healthcare facility type'].isna()& df['Year_with_nans'].isna()]

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
1557549,NaN,NaN,NaN,NaN,NaN,I702,16,1,2022
1557550,NaN,NaN,NaN,NaN,NaN,N764,9,1,2022
1557551,NaN,NaN,NaN,NaN,NaN,S299,5,1,2022
1557552,NaN,NaN,NaN,NaN,NaN,P220,6,1,2022
1557553,NaN,NaN,NaN,NaN,NaN,P220,20,1,2022
...,...,...,...,...,...,...,...,...,...
1612262,NaN,NaN,NaN,NaN,NaN,O244,2,1,2023
1612263,NaN,NaN,NaN,NaN,NaN,K811,1,1,2023
1612264,NaN,NaN,NaN,NaN,NaN,P034,2,1,2023
1612265,NaN,NaN,NaN,NaN,NaN,P034,2,1,2023


In [19]:
df[df['Healthcare facility type'].isna()& df['Age'].isna()]

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
1557549,NaN,NaN,NaN,NaN,NaN,I702,16,1,2022
1557550,NaN,NaN,NaN,NaN,NaN,N764,9,1,2022
1557551,NaN,NaN,NaN,NaN,NaN,S299,5,1,2022
1557552,NaN,NaN,NaN,NaN,NaN,P220,6,1,2022
1557553,NaN,NaN,NaN,NaN,NaN,P220,20,1,2022
...,...,...,...,...,...,...,...,...,...
1612262,NaN,NaN,NaN,NaN,NaN,O244,2,1,2023
1612263,NaN,NaN,NaN,NaN,NaN,K811,1,1,2023
1612264,NaN,NaN,NaN,NaN,NaN,P034,2,1,2023
1612265,NaN,NaN,NaN,NaN,NaN,P034,2,1,2023


In [20]:
# Also, apart of that all the missing values for healthcare facility that are in 2023, there are not valid values in that year.
# Maybe there was a policy decision? But this is not random
df[df['Healthcare facility type'].notna()& df['Year_with_nans'].notna()]["Year_with_nans"].value_counts()

Year_with_nans
2024    1667349
2022    1557549
2021    1467062
2023          0
Name: count, dtype: int64

In [21]:
# The proportion of men and women when the healthcare facility type is missing follow an usual pattern, where there are more women in hospital
df[df['Healthcare facility type'].isna()].Sex.value_counts()

Sex
MUJER                        854914
HOMBRE                       629245
1                                 0
2                                 0
3                                 0
9                                 0
INTERSEX (INDETERMINDADO)         0
Name: count, dtype: int64

In [22]:
# The healthcare insurance also follow an usual pattern, where Fonasa is more frequent than Isapre
df[df['Healthcare facility type'].isna()]["Health insurance"].value_counts()

Health insurance
FONASA         1148342
ISAPRE          275904
DESCONOCIDO      16496
DIPRECA          13588
CAPREDENA        11553
NINGUNA          10170
SISA              8106
Name: count, dtype: int64

In [23]:
nan_facility_type_age = df[df['Healthcare facility type'].isna()]["Age"].value_counts(normalize=True) * 100
nan_facility_type_age

Age
30 a 39                     16.185597
60 a 69                     13.870818
20 a 29                     12.336347
50 a 59                     11.689516
70 a 79                     11.439340
40 a 49                     11.325539
80 a 89                      6.382133
1 a 9                        6.133844
10 a 19                      5.865477
menor de un año              3.350248
90 y más                     1.421142
60 A 64 AÑOS                 0.000000
80 A 84 AÑOS                 0.000000
70 A 74 AÑOS                 0.000000
7 A 27 DIAS                  0.000000
85 A MAS                     0.000000
65 A 69 AÑOS                 0.000000
menor a 7 días               0.000000
75 A 79 AÑOS                 0.000000
1 A 4 AÑOS                   0.000000
55 A 59 AÑOS                 0.000000
50 A 54 AÑOS                 0.000000
45 A 49 AÑOS                 0.000000
40 A 44 AÑOS                 0.000000
35 A 39 AÑOS                 0.000000
30 A 34 AÑOS                 0.000000
28 DIAS 

In [24]:
all_facility_type_age = df.Age.value_counts(normalize=True) * 100
all_facility_type_age

Age
30 a 39                     8.160044
60 a 69                     6.658372
20 a 29                     6.282068
50 a 59                     5.767959
40 a 49                     5.639415
70 a 79                     5.460598
30 A 34 AÑOS                4.468923
35 A 39 AÑOS                3.951608
25 A 29 AÑOS                3.699864
60 A 64 AÑOS                3.478706
65 A 69 AÑOS                3.378044
55 A 59 AÑOS                3.264720
40 A 44 AÑOS                3.113460
80 a 89                     3.064416
70 A 74 AÑOS                2.981549
1 a 9                       2.914144
50 A 54 AÑOS                2.892885
45 A 49 AÑOS                2.840975
10 a 19                     2.831455
20 A 24 AÑOS                2.635150
75 A 79 AÑOS                2.609341
85 A MAS                    1.997031
80 A 84 AÑOS                1.936961
menor de un año             1.745692
15 A 19 AÑOS                1.636675
1 A 4 AÑOS                  1.449308
5 A 9 AÑOS                  1.4029

In [25]:
# The age groups overlap with each other. For example "Age 30 to 39" includes "Age 30 to 34" and "Age 35 to 39"
# For some reason this overlapping does not happen when the healtcare facility type is nan. See the cell below for more details.
# The way the data was grouped shows different percents for healthcare facility type when is valid or NaN, but in reality are quite similar.
# So the question is: is there a reason behind of grouping the age of the people differently when the healthcare facility type is valid compared to NaN?
# When the column "Age" is cleaned, this is going to be clearer
pd.DataFrame({'all_facility_type_age': all_facility_type_age, 'nan_facility_type_age': nan_facility_type_age}).sort_values(by=['all_facility_type_age'], ascending = False)

,all_facility_type_age,nan_facility_type_age
Age,,
30 a 39,8.160044,16.185597
60 a 69,6.658372,13.870818
20 a 29,6.282068,12.336347
50 a 59,5.767959,11.689516
40 a 49,5.639415,11.325539
70 a 79,5.460598,11.439340
30 A 34 AÑOS,4.468923,0.000000
35 A 39 AÑOS,3.951608,0.000000
25 A 29 AÑOS,3.699864,0.000000


In [26]:
# Example:

# Valid heathcare facility types for Age 30-39, 30-34, 35-39, respectively
print(8.160044 + 4.468923 + 3.951608)
# NaN heathcare facility types for Age 30-39, 30-34, 35-39, respectively
print(16.185597 + 0 + 0)

16.580575
16.185597


Sex

In [28]:
# There are different ways to label each sex, so a function is defined to fix that and consier just men and women
df["Sex"].value_counts()

Sex
2                            1789953
MUJER                        1761041
1                            1338190
HOMBRE                       1280652
3                                 49
INTERSEX (INDETERMINDADO)         15
9                                  4
Name: count, dtype: int64

In [29]:
  def sexo(x):
        if x == "2" or x == "MUJER":
            x = "Mujer"
        elif x == "1" or x == "HOMBRE":
            x = "Hombre"
        else:
            x = "Otro"
        
        return x

In [30]:
# Function is applied over the column
df["Sex"] = df["Sex"].apply(sexo)

In [31]:
# Just men and women are kept in the dataframe
df = df[df['Sex'] != 'Otro']

In [32]:
# There are more women than men, as usual
df["Sex"].value_counts()

Sex
Mujer     3550994
Hombre    2618842
Name: count, dtype: int64

In [33]:
# is there another variable related to the missingness in sex?

In [34]:
# Apparently the missingess is structured, so this will be explored
df.loc[df['Sex'].isnull()].isna().sum()

Healthcare facility type    167677
Sex                         173892
Age                         167677
Health insurance            173892
Year_with_nans              167677
Primary diagnosis code           0
Length of stay                   0
Discharge condition              0
Year                             0
dtype: int64

In [35]:
# A dataframe that contains only rows when value column in sex is missing is created
nan_sex = df.loc[df['Sex'].isnull()]

In [36]:
nan_sex

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
1557549,NaN,NaN,NaN,NaN,NaN,I702,16,1,2022
1557550,NaN,NaN,NaN,NaN,NaN,N764,9,1,2022
1557551,NaN,NaN,NaN,NaN,NaN,S299,5,1,2022
1557552,NaN,NaN,NaN,NaN,NaN,P220,6,1,2022
1557553,NaN,NaN,NaN,NaN,NaN,P220,20,1,2022
...,...,...,...,...,...,...,...,...,...
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P073,4,1,2024
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P590,2,1,2024
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P599,2,1,2024
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,menor a 7 días,NaN,2024,E250,52,1,2024


In [37]:
# loop over the columns to count the values on each one
# Age, year_with_nans and healthcare facility type have 6215 values that are valid when in sex are NaN. In year this is 2024
for col in nan_sex.columns:
    print(f"Value counts for '{col}':")
    print(nan_sex[col].value_counts(dropna = False))
    print("\n" + "-"*40 + "\n")

Value counts for 'Healthcare facility type':
Healthcare facility type
NaN                                                                  167677
No Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS      4620
Pertenecientes al Sistema Nacional de Servicios de Salud, SNSS         1595
Name: count, dtype: int64

----------------------------------------

Value counts for 'Sex':
Sex
NaN    173892
Name: count, dtype: int64

----------------------------------------

Value counts for 'Age':
Age
NaN                         167677
65 A 69 AÑOS                   528
30 A 34 AÑOS                   480
60 A 64 AÑOS                   471
70 A 74 AÑOS                   459
35 A 39 AÑOS                   450
40 A 44 AÑOS                   393
45 A 49 AÑOS                   359
50 A 54 AÑOS                   359
55 A 59 AÑOS                   353
25 A 29 AÑOS                   345
75 A 79 AÑOS                   273
20 A 24 AÑOS                   269
menor a 7 días                 259
85 A

In [38]:
# Total valid values for column "Age" and "Healthcare facility type" when "Sex" is not valid
print(nan_sex["Age"].value_counts(dropna = True).sum())
print(nan_sex["Healthcare facility type"].value_counts(dropna = True).sum())

6215
6215


When sex is NaN this is the summary of what happens in the other columns:

1. Age: There are 6215 valid values distributed among the different age ranges. The rest are NaNs
2. Healthcare facility type: There are 6215 valid values. The rest are NaNs, also when the year is 2023
3. Health insurance: if the sex is not valid, the same happens with the health insurance. 0 valid values here and all NaNs
4. Year_with_nans: There are 6215 valid values only for year 2024. The rest are NaNs
5. Primary diagnosis code, Length of stay and discharge condition: All valid values beacuse there are not NaNs in the whole dataframe
6. Year (column created by me based on the year of the dataframe): 6215 values for 2024 and all the values in 2021 are valid, so is not present in this dataframe, with only NaN values for sex. The missingness of the data will not be analyzed with this column as it was created by me

Age

In [41]:
# There are many age groups and some overlap with each other, so it is necessary tu customize it
df["Age"].value_counts()

Age
30 a 39                     503974
60 a 69                     411229
20 a 29                     387988
50 a 59                     356236
40 a 49                     348297
70 a 79                     337253
30 A 34 AÑOS                276004
35 A 39 AÑOS                244056
25 A 29 AÑOS                228508
60 A 64 AÑOS                214849
65 A 69 AÑOS                208632
55 A 59 AÑOS                201633
40 A 44 AÑOS                192291
80 a 89                     189262
70 A 74 AÑOS                184144
1 a 9                       179981
50 A 54 AÑOS                178668
45 A 49 AÑOS                175462
10 a 19                     174869
20 A 24 AÑOS                162748
75 A 79 AÑOS                161156
85 A MAS                    123338
80 A 84 AÑOS                119629
menor de un año             107806
15 A 19 AÑOS                101076
1 A 4 AÑOS                   89511
5 A 9 AÑOS                   86648
10 A 14 AÑOS                 81169
menor a 7 días  

In [42]:
# Remember this is the number of NaN values.
# According to the previous findings, the rows are NaN in 'Healthcare facility type', 'Sex', Health insurance', 'Year_with_nans' and 'Age' for some reason
# Also there are 6215 values that are valid for 'Age', 'Healthcare facility type' and 'Health insurance', while NaN in the rest of the columns mentioned
df["Age"].isna().sum()

167677

In [43]:
# A quick look of the data, whose dataframe is the same as in 'years_with_nans'. Look cell below
nan_age = df.loc[df['Age'].isnull()]
nan_years = df.loc[df["Year_with_nans"].isnull()]

In [44]:
# Dataframes are identic
nan_age.equals(nan_years)

True

In [45]:
nan_age.shape

(167677, 9)

In [46]:
nan_age.head()

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
1557549,NaN,NaN,NaN,NaN,NaN,I702,16,1,2022
1557550,NaN,NaN,NaN,NaN,NaN,N764,9,1,2022
1557551,NaN,NaN,NaN,NaN,NaN,S299,5,1,2022
1557552,NaN,NaN,NaN,NaN,NaN,P220,6,1,2022
1557553,NaN,NaN,NaN,NaN,NaN,P220,20,1,2022


# Valid values column Age

In [48]:
# Back to the valid values in the column
age_groups = list(df["Age"].unique())

In [49]:
age_groups

['1 A 4 AÑOS',
 '10 A 14 AÑOS',
 '15 A 19 AÑOS',
 '2 MESES A MENOS DE 1 AÑO',
 '20 A 24 AÑOS',
 '25 A 29 AÑOS',
 '28 DIAS A 2 MES',
 '30 A 34 AÑOS',
 '35 A 39 AÑOS',
 '40 A 44 AÑOS',
 '45 A 49 AÑOS',
 '5 A 9 AÑOS',
 '50 A 54 AÑOS',
 '55 A 59 AÑOS',
 '60 A 64 AÑOS',
 '65 A 69 AÑOS',
 '7 A 27 DIAS',
 '70 A 74 AÑOS',
 '75 A 79 AÑOS',
 '80 A 84 AÑOS',
 '85 A MAS',
 'menor a 7 días',
 '1 a 9',
 '10 a 19',
 '20 a 29',
 '30 a 39',
 '40 a 49',
 '50 a 59',
 '60 a 69',
 '70 a 79',
 '80 a 89',
 '90 y más',
 'menor de un año',
 nan]

In [50]:
  def age(x):
        if x == "menor de un año" or x == "menor a 7 días" or x == "2 MESES A MENOS DE 1 AÑO" or x == "7 A 27 DIAS" or x == "28 DIAS A 2 MES":
            x = "0 a 1"
        elif x == "90 y más":
            x = "90 a más"
        elif x == "85 A MAS":
            x = "85 a más"
        return x

In [51]:
df["Age"] = df["Age"].apply(age)
age_groups = list(df["Age"].unique())
df["Age"] = df['Age'].str.lower().str.replace('años', '').str.rstrip()
age_groups_updated = list(df["Age"].unique())

In [52]:
age_groups_updated

['1 a 4',
 '10 a 14',
 '15 a 19',
 '0 a 1',
 '20 a 24',
 '25 a 29',
 '30 a 34',
 '35 a 39',
 '40 a 44',
 '45 a 49',
 '5 a 9',
 '50 a 54',
 '55 a 59',
 '60 a 64',
 '65 a 69',
 '70 a 74',
 '75 a 79',
 '80 a 84',
 '85 a más',
 '1 a 9',
 '10 a 19',
 '20 a 29',
 '30 a 39',
 '40 a 49',
 '50 a 59',
 '60 a 69',
 '70 a 79',
 '80 a 89',
 '90 a más',
 nan]

In [53]:
df["Age"] = df["Age"].str.split("a")

In [54]:
ages_list = df["Age"].dropna()

In [55]:
# Step: Keep only rows where 'más' is exactly one of the elements in the list
ages_with_mas = ages_list[ages_list.apply(lambda x: x[0].strip() == 'más' or x[1].strip() == 'más')]


In [56]:
ages_with_mas

254675     [85 ,  más]
254676     [85 ,  más]
254677     [85 ,  más]
254678     [85 ,  más]
254679     [85 ,  más]
              ...     
1667156    [85 ,  más]
1667157    [85 ,  más]
1667158    [85 ,  más]
1667159    [85 ,  más]
1667160    [85 ,  más]
Name: Age, Length: 168136, dtype: object

In [57]:
def convert_age_range(age_list):
    start = int(age_list[0].strip())
    end_raw = age_list[1].strip()
    end = 120 if end_raw == 'más' else int(end_raw)
    return [start, end]

ages_final = ages_list.apply(convert_age_range)


In [58]:
ages_final

0          [1, 4]
1          [1, 4]
2          [1, 4]
3          [1, 4]
4          [1, 4]
            ...  
1667344    [0, 1]
1667345    [0, 1]
1667346    [0, 1]
1667347    [0, 1]
1667348    [0, 1]
Name: Age, Length: 6176051, dtype: object

In [59]:
# Step 0: Save original index
original_index = df.index

# Step 1: Reset index to have a unique one for safe operations
df_reset = df.reset_index(drop=True)

# Step 2: Drop NaNs in Age column and transform those rows
ages_list = df_reset["Age"].dropna()
ages_final = ages_list.apply(convert_age_range)

# Step 3: Create a new empty column in df_reset
df_reset["Age_range"] = pd.NA

# Step 4: Assign transformed values back to the matching rows by index
df_reset.loc[ages_final.index, "Age_range"] = ages_final

# Step 5: Restore original index to df_reset
df_reset.index = original_index

# Optional: If you want, assign df_reset back to df or keep working with df_reset
df = df_reset


In [60]:
# Replace missing Age_range with [pd.NA, pd.NA] to avoid errors
df['Age_range_filled'] = df['Age_range'].apply(lambda x: x if isinstance(x, list) else [pd.NA, pd.NA])

# Now split into two columns
df[['age_min', 'age_max']] = pd.DataFrame(df['Age_range_filled'].tolist(), index=df.index)


In [61]:
df

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age_range,Age_range_filled,age_min,age_max
0,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",ISAPRE,2021,K590,1,1,2021,"[1, 4]","[1, 4]",1,4
1,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",NINGUNA,2021,S015,1,1,2021,"[1, 4]","[1, 4]",1,4
2,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",ISAPRE,2021,T181,1,1,2021,"[1, 4]","[1, 4]",1,4
3,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",FONASA,2021,Q381,1,1,2021,"[1, 4]","[1, 4]",1,4
4,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",FONASA,2021,Q531,1,1,2021,"[1, 4]","[1, 4]",1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,"[0 , 1]",NaN,2024,P073,4,1,2024,"[0, 1]","[0, 1]",0,1
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,"[0 , 1]",NaN,2024,P590,2,1,2024,"[0, 1]","[0, 1]",0,1
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,"[0 , 1]",NaN,2024,P599,2,1,2024,"[0, 1]","[0, 1]",0,1
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,"[0 , 1]",NaN,2024,E250,52,1,2024,"[0, 1]","[0, 1]",0,1


In [62]:
df["Age_range_filled"].value_counts()

Age_range_filled
[30, 39]        503974
[60, 69]        411229
[20, 29]        387988
[50, 59]        356236
[40, 49]        348297
[70, 79]        337253
[30, 34]        276004
[35, 39]        244056
[25, 29]        228508
[60, 64]        214849
[0, 1]          212642
[65, 69]        208632
[55, 59]        201633
[40, 44]        192291
[80, 89]        189262
[70, 74]        184144
[1, 9]          179981
[50, 54]        178668
[45, 49]        175462
[10, 19]        174869
[<NA>, <NA>]    167677
[20, 24]        162748
[75, 79]        161156
[85, 120]       123338
[80, 84]        119629
[15, 19]        101076
[1, 4]           89511
[5, 9]           86648
[10, 14]         81169
[90, 120]        44798
Name: count, dtype: int64

In [63]:
df_age = df.copy()

In [64]:
def fix_age_ranges(x):
    # lista dentro de la lista
    age_range_list = [[0, 1], [1, 9], [10, 19], [20, 29], [30, 39], [40, 49], [50, 59], [60, 69], [70, 79], [80, 120]]
    # valor en age_range_filled
    rango_evaluado = x
    new_range = rango_evaluado.copy()
    # Lógica de la función
    for l in age_range_list:
        if pd.isna(rango_evaluado[0]) or pd.isna(rango_evaluado[1]):
            continue
        elif rango_evaluado[0] >= l[0] and rango_evaluado[1] <= l[1]:
            rango_evaluado = l
            new_range = [rango_evaluado[0], rango_evaluado[1]]
            break
    return new_range

In [65]:
df_age["Age_range_filled"] = df_age["Age_range_filled"].apply(fix_age_ranges)

In [66]:
df_age["Age_range_filled"].value_counts()

Age_range_filled
[30, 39]        1024034
[60, 69]         834710
[20, 29]         779244
[50, 59]         736537
[40, 49]         716050
[70, 79]         682553
[80, 120]        477027
[10, 19]         357114
[1, 9]           356140
[0, 1]           212642
[<NA>, <NA>]     167677
Name: count, dtype: int64

In [67]:
# Split the updated 'Age_range_filled' into two columns directly
df_age[['age_min', 'age_max']] = pd.DataFrame(df_age['Age_range_filled'].tolist(), index=df_age.index)

In [68]:
df = df_age

In [69]:
df.head()

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age_range,Age_range_filled,age_min,age_max
0,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",ISAPRE,2021,K590,1,1,2021,"[1, 4]","[1, 9]",1,9
1,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",NINGUNA,2021,S015,1,1,2021,"[1, 4]","[1, 9]",1,9
2,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",ISAPRE,2021,T181,1,1,2021,"[1, 4]","[1, 9]",1,9
3,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",FONASA,2021,Q381,1,1,2021,"[1, 4]","[1, 9]",1,9
4,No Pertenecientes al Sistema Nacional de Servi...,Hombre,"[1 , 4]",FONASA,2021,Q531,1,1,2021,"[1, 4]","[1, 9]",1,9


In [70]:
df = df.drop(columns=['Age_range', 'age_min', "age_max", "Age"])

In [71]:
df.rename(columns={'Age_range_filled': 'Age'}, inplace=True)

In [171]:
df['Age'] = df['Age'].apply(lambda x: pd.NA if isinstance(x, list) and all(pd.isna(i) for i in x) else x)

In [72]:
df.tail()

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P073,4,1,2024,"[0, 1]"
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P590,2,1,2024,"[0, 1]"
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P599,2,1,2024,"[0, 1]"
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,E250,52,1,2024,"[0, 1]"
1667348,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P073,2,2,2024,"[0, 1]"


# RETOMAR AQUI: VER SI FALTA ALGO POR AHCER CON AGE ANTES DE PASAR A LA OTRA VARIABLE

In [74]:
df["Health insurance"].value_counts()

Health insurance
FONASA         4623950
ISAPRE         1205759
DESCONOCIDO      86854
DIPRECA          78281
NINGUNA          77315
CAPREDENA        56787
SISA             40890
Name: count, dtype: int64

In [145]:
  def insurance(x):
        if x == "FONASA":
            x = "Fonasa"
        elif x == "ISAPRE":
            x = "Isapre"
        else:
            x = "Eliminar"
        
        return x

In [149]:
df["Health insurance"] = df["Health insurance"].apply(insurance)

In [161]:
df = df[df["Health insurance"]!= "Eliminar"]

In [175]:
df.loc[df["Health insurance"].isnull()].isna().sum()

Healthcare facility type    167677
Sex                         173892
Health insurance            173892
Year_with_nans              167677
Primary diagnosis code           0
Length of stay                   0
Discharge condition              0
Year                             0
Age                         167677
dtype: int64

In [177]:
nan_insurance = df.loc[df['Health insurance'].isnull()]

In [185]:
nan_sex

,Healthcare facility type,Sex,Age,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year
1557549,NaN,NaN,NaN,NaN,NaN,I702,16,1,2022
1557550,NaN,NaN,NaN,NaN,NaN,N764,9,1,2022
1557551,NaN,NaN,NaN,NaN,NaN,S299,5,1,2022
1557552,NaN,NaN,NaN,NaN,NaN,P220,6,1,2022
1557553,NaN,NaN,NaN,NaN,NaN,P220,20,1,2022
...,...,...,...,...,...,...,...,...,...
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P073,4,1,2024
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P590,2,1,2024
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,7 A 27 DIAS,NaN,2024,P599,2,1,2024
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,menor a 7 días,NaN,2024,E250,52,1,2024


In [187]:
nan_insurance

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
1557549,NaN,NaN,NaN,NaN,I702,16,1,2022,<NA>
1557550,NaN,NaN,NaN,NaN,N764,9,1,2022,<NA>
1557551,NaN,NaN,NaN,NaN,S299,5,1,2022,<NA>
1557552,NaN,NaN,NaN,NaN,P220,6,1,2022,<NA>
1557553,NaN,NaN,NaN,NaN,P220,20,1,2022,<NA>
...,...,...,...,...,...,...,...,...,...
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P073,4,1,2024,"[0, 1]"
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P590,2,1,2024,"[0, 1]"
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P599,2,1,2024,"[0, 1]"
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,E250,52,1,2024,"[0, 1]"


Year_with_nans

In [230]:
df["Year_with_nans"].value_counts(dropna = False)

Year_with_nans
2024    1574632
2022    1465952
2023    1424246
2021    1371094
NaN      167677
Name: count, dtype: int64

In [218]:
# Recordar que sex y health insurance tienen 6125 valores nulos extra (o health facility type, yerar with nans y age tienen 6125 valores validos extra)
# Depende del punto de vista, pero todos comparten 167.677 valores nulos
df.loc[df["Year_with_nans"].isnull()].isna().sum()

Healthcare facility type    167677
Sex                         167677
Health insurance            167677
Year_with_nans              167677
Primary diagnosis code           0
Length of stay                   0
Discharge condition              0
Year                             0
Age                         167677
dtype: int64

In [ ]:
# In the rest of the variables there are not missing values, so the date will be merged with the discharges from 2001 to 2021 to be analyzed

In [222]:
df["Primary diagnosis code"].value_counts()

Primary diagnosis code
U071    112660
O800    110127
O820     91017
K358     85407
K802     82887
         ...  
R261         0
Q274         0
J671         0
R789         0
B669         0
Name: count, Length: 7872, dtype: int64

In [224]:
df["Length of stay"].value_counts()

Length of stay
1       1950043
2       1114587
3        665994
4        413971
5        294661
         ...   
1079          1
1034          1
1662          1
753           1
2611          1
Name: count, Length: 1339, dtype: int64

In [226]:
df["Discharge condition"].value_counts()

Discharge condition
1    5844040
2     159561
Name: count, dtype: int64

In [228]:
df["Year"].value_counts()

Year
2024    1574632
2023    1552354
2022    1505521
2021    1371094
Name: count, dtype: int64

# Handling duplicates

In [237]:
df_duplicates = df.copy()

In [243]:
df_duplicates['Age'] = df_duplicates['Age'].apply(lambda x: tuple(x) if type(x) is list else x)

In [253]:
# Considering there are not patient IDs or hospital name linked to each row is not possible to make a distinction between data entry issues and real patients
# when rows are duplicated, so no data will be deleted to avoid losing valuable data
df_duplicates[df_duplicates.duplicated(keep=False)]

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
0,No Pertenecientes al Sistema Nacional de Servi...,Hombre,Isapre,2021,K590,1,1,2021,"(1, 9)"
2,No Pertenecientes al Sistema Nacional de Servi...,Hombre,Isapre,2021,T181,1,1,2021,"(1, 9)"
3,No Pertenecientes al Sistema Nacional de Servi...,Hombre,Fonasa,2021,Q381,1,1,2021,"(1, 9)"
4,No Pertenecientes al Sistema Nacional de Servi...,Hombre,Fonasa,2021,Q531,1,1,2021,"(1, 9)"
5,No Pertenecientes al Sistema Nacional de Servi...,Hombre,Fonasa,2021,Q539,1,1,2021,"(1, 9)"
...,...,...,...,...,...,...,...,...,...
1667335,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,C189,2,1,2024,"(40, 49)"
1667336,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,C189,2,1,2024,"(40, 49)"
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P073,4,1,2024,"(0, 1)"
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P590,2,1,2024,"(0, 1)"


In [257]:
df_duplicates.duplicated().sum() / len(df) * 100

74.65321229708637

In [259]:
df_duplicates.value_counts().value_counts()


count
1       744294
2       156171
3        68069
4        37578
5        23982
         ...  
678          1
677          1
674          1
671          1
7634         1
Name: count, Length: 822, dtype: int64

In [270]:
# Many duplicates are in the birth and they will be deleted as the do not represent an illness, just a natural circunstance
row_counts = df_duplicates.value_counts()
rows_x_times = row_counts[row_counts == 671]
df_duplicates[df_duplicates.apply(tuple, axis=1).isin(rows_x_times.index)]

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
26459,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
26686,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
50504,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
98038,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
98262,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
...,...,...,...,...,...,...,...,...,...
1641541,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
1641584,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
1641592,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"
1641596,Pertenecientes al Sistema Nacional de Servicio...,Mujer,Fonasa,2024,O820,3,1,2024,"(20, 29)"


# despues de eliminar datos validos (ej. sexos que no importan) comparar df_6215, df_167677, df_1484159 y df_validos considerando las variables diagnosis code, length of stay y discharge condition



In [274]:
df.isna().sum()

Healthcare facility type    1591923
Sex                          173892
Health insurance             173892
Year_with_nans               167677
Primary diagnosis code            0
Length of stay                    0
Discharge condition               0
Year                              0
Age                          167677
dtype: int64

# Columns Primary diagnosis code, Length of stay and Discharge condition analysis

Dataframes to be created to compare valid values in Primary diagnosis code, Length of stay and Discharge condition columns

-> only valid values in the dataframe
-> when 167677 are nan in all the rest of the columns
-> when 6215 are valid for Age and Year_with_nans but NaN in Sex and Health insurance
-> when only are NaN in Healthcare facility type (it matches that the year_with_nans is always 2023)

Only valid values in the dataframe

In [284]:
df_pld_valid = df.copy()

In [286]:
df_pld_valid = df_pld.dropna()

When 167677 are nan in all the rest of the columns

In [291]:
df_167677 = df.copy()

In [301]:
df_167677 = df_167677[df_167677["Age"].isna() & df_167677["Sex"].isna() & df_167677["Healthcare facility type"].isna() & df_167677["Health insurance"].isna() & df_167677["Year_with_nans"].isna()]

When 6215 are valid for Age and Year_with_nans but NaN in Sex and Health insurance

In [319]:
df_6215 = df.copy()

In [321]:
df_6215[df_6215["Age"].notna() & df_6215["Sex"].isna() & df_6215["Health insurance"].isna() & df_6215["Year_with_nans"].notna()]

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
1661134,No Pertenecientes al Sistema Nacional de Servi...,NaN,NaN,2024,J189,2,1,2024,"[1, 9]"
1661135,No Pertenecientes al Sistema Nacional de Servi...,NaN,NaN,2024,J209,2,1,2024,"[1, 9]"
1661136,No Pertenecientes al Sistema Nacional de Servi...,NaN,NaN,2024,J353,1,1,2024,"[1, 9]"
1661137,No Pertenecientes al Sistema Nacional de Servi...,NaN,NaN,2024,Z470,1,1,2024,"[1, 9]"
1661138,No Pertenecientes al Sistema Nacional de Servi...,NaN,NaN,2024,R683,1,1,2024,"[1, 9]"
...,...,...,...,...,...,...,...,...,...
1667344,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P073,4,1,2024,"[0, 1]"
1667345,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P590,2,1,2024,"[0, 1]"
1667346,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,P599,2,1,2024,"[0, 1]"
1667347,Pertenecientes al Sistema Nacional de Servicio...,NaN,NaN,2024,E250,52,1,2024,"[0, 1]"


When only are NaN in Healthcare facility type (it matches that the year_with_nans is always 2023)

In [ ]:
167677 (year or age is missing) + 1484159 (year is 2023) = 1651836 (total amount of NaNs for healthcare facility type column)

In [323]:
df_facility_type = df.copy()

In [332]:
mask1 = df_facility_type["Healthcare facility type"].isna()
mask2 = df_facility_type["Year_with_nans"] == "2023"
df_facility_type[mask1 & mask2]

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
0,NaN,Hombre,Fonasa,2023,D180,7,1,2023,"[1, 9]"
1,NaN,Hombre,Fonasa,2023,T181,2,1,2023,"[1, 9]"
2,NaN,Hombre,Fonasa,2023,K358,1,1,2023,"[1, 9]"
3,NaN,Hombre,Fonasa,2023,K353,6,1,2023,"[1, 9]"
4,NaN,Hombre,Fonasa,2023,K353,1,1,2023,"[1, 9]"
...,...,...,...,...,...,...,...,...,...
1484154,NaN,Hombre,Fonasa,2023,Q249,2,1,2023,"[0, 1]"
1484155,NaN,Hombre,Fonasa,2023,D807,3,1,2023,"[0, 1]"
1484156,NaN,Hombre,Fonasa,2023,A499,4,1,2023,"[0, 1]"
1484157,NaN,Hombre,Fonasa,2023,T182,1,1,2023,"[0, 1]"


In [334]:
df_facility_type[df_facility_type["Age"].notna() & df_facility_type["Sex"].notna() & df_facility_type["Health insurance"].notna() & df_facility_type["Year_with_nans"].notna() & df_facility_type["Healthcare facility type"].isna()]

,Healthcare facility type,Sex,Health insurance,Year_with_nans,Primary diagnosis code,Length of stay,Discharge condition,Year,Age
0,NaN,Hombre,Fonasa,2023,D180,7,1,2023,"[1, 9]"
1,NaN,Hombre,Fonasa,2023,T181,2,1,2023,"[1, 9]"
2,NaN,Hombre,Fonasa,2023,K358,1,1,2023,"[1, 9]"
3,NaN,Hombre,Fonasa,2023,K353,6,1,2023,"[1, 9]"
4,NaN,Hombre,Fonasa,2023,K353,1,1,2023,"[1, 9]"
...,...,...,...,...,...,...,...,...,...
1484154,NaN,Hombre,Fonasa,2023,Q249,2,1,2023,"[0, 1]"
1484155,NaN,Hombre,Fonasa,2023,D807,3,1,2023,"[0, 1]"
1484156,NaN,Hombre,Fonasa,2023,A499,4,1,2023,"[0, 1]"
1484157,NaN,Hombre,Fonasa,2023,T182,1,1,2023,"[0, 1]"


In [336]:
df_facility_type[mask1 & mask2].equals(df_facility_type[df_facility_type["Age"].notna() & df_facility_type["Sex"].notna() & df_facility_type["Health insurance"].notna() & df_facility_type["Year_with_nans"].notna() & df_facility_type["Healthcare facility type"].isna()])

True

# Tomar todos los dataframes y comparar las 3 columnas clave

# Tambien comprobar si los 6215 valores son iguales (en Age, year_with_nans y healthcare facility type) sin importar que variable estoy viendo su nulidad (sex o health insurance()

# insights por escalerecer:
1. Porque healthcare facility tiene tantos NaN el año 2023?
2. Por qué la agrupación de pacientes según año es mejor cuando healthcare facility type es nulo?

# Recordar describir con comentarios los pasos en cada cell